In [32]:

from os import listdir
from os.path import isfile, join
import json
# Put the JHU Repo here for local access
repo = '/home/jovyan/COVID-19'

path = repo + '/csse_covid_19_data/csse_covid_19_daily_reports'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f)) and f.endswith('csv')]
months = [(3, 21, 31), (4, 1, 30), (5, 1, 31), (6, 1, 30), (7, 1, 10)]
db = [['month', 'day', 'state', 'county', 'cases','deaths', 'recovered', 'active']]

def has_data(month, day):
    # county-level data is only available from 3/21/2020 on
    return month > 3 or (month == 3 and day > 21)

# Data cleansing
def convert_to_int(str):
    try:
        return int(str)
    except ValueError:
        return 0

#  read a file, and for each line append it to the db
def process_file(file_name):
    month = int(file_name[:2])
    day = int(file_name[3:5])
    date = {'month': month, 'day': day}
    if has_data(month, day):
        file = open(path + '/' + file_name, 'r')
        lines = file.readlines()[1:]
        file.close()
        records = [line.split(',') for line in lines]
        for record in records:
            db.append([month, day, record[2],  record[1],  convert_to_int(record[7]), convert_to_int(record[8]), convert_to_int(record[9]), convert_to_int(record[10])])

# process_file('07-06-2020.csv')
# db

# process every file
for f in onlyfiles: process_file(f)  

# write the JSON
import json
out = json.dumps(db)
# dump it to data/covid-reports.json
f = open('data/covid-reports.json', 'w')
f.write(out)
f.close()